In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math



In [2]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)  # Create an empty tensor to store positional encodings.
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # Create a range of positions.
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))  # Scaling factor for positions.
        pe[:, 0::2] = torch.sin(position * div_term)  # Apply sine to even positions.
        pe[:, 1::2] = torch.cos(position * div_term)  # Apply cosine to odd positions.
        pe = pe.unsqueeze(0).transpose(0, 1)  # Add an extra dimension for batch and transpose dimensions.
        self.register_buffer('pe', pe)  # Store positional encodings in a buffer so it's not trainable.

    def forward(self, x):
        return x + self.pe[:x.size(0), :]  # Add positional encoding to input x.


In [3]:
class TransformerModel(nn.Module):
    def __init__(self, ntoken, d_model, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model)  # Use the positional encoding we defined earlier.
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, nhid, dropout)  # One layer of the transformer encoder.
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)  # Stack multiple layers of the encoder.
        self.encoder = nn.Embedding(ntoken, d_model)  # Embedding layer to convert tokens (words) to vectors.
        self.d_model = d_model  # Dimension of model.
        self.decoder = nn.Linear(d_model, ntoken)  # Linear layer to convert the output back to token predictions.

        self.init_weights()  # Initialize weights for the encoder and decoder.

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)  # Initialize embedding weights.
        self.decoder.bias.data.zero_()  # Initialize decoder biases to zero.
        self.decoder.weight.data.uniform_(-initrange, initrange)  # Initialize decoder weights.

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.d_model)  # Pass input through embedding layer and scale.
        src = self.pos_encoder(src)  # Add positional encoding.
        output = self.transformer_encoder(src, src_mask)  # Pass through transformer encoder.
        output = self.decoder(output)  # Pass through decoder to get final predictions.
        return output


In [4]:
# Hyperparameters
ntokens = 10000  # size of vocabulary
emsize = 200  # embedding dimension
nhid = 200  # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # the number of heads in the multiheadattention models
dropout = 0.2  # the dropout value



In [5]:
# Create the model
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout)



/home/codespace/.local/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
src = torch.randint(0, ntokens, (10, 32))  # A random batch of input sequences (sequence_length=10, batch_size=32)
src_mask = torch.zeros((10, 10)).type(torch.bool)  # Mask to prevent looking at future tokens


In [7]:
# Forward pass
output = model(src, src_mask)
print(output.shape)  # Should be (10, 32, ntokens)



torch.Size([10, 32, 10000])


In [8]:
criterion = nn.CrossEntropyLoss()  # Loss function used to measure how good the model’s predictions are.
optimizer = optim.Adam(model.parameters())  # Adam optimizer, used to update the model’s weights.


In [9]:
for epoch in range(10):  # Loop over epochs
    model.train()  # Set model to training mode
    for batch in range(100):  # Assume 100 batches of training data
        optimizer.zero_grad()  # Reset gradients
        output = model(src, src_mask)  # Forward pass
        loss = criterion(output.view(-1, ntokens), src.view(-1))  # Calculate loss
        loss.backward()  # Backpropagate
        optimizer.step()  # Update model weights
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 0.011590269394218922
Epoch 2, Loss: 0.005491065792739391
Epoch 3, Loss: 0.0032924532424658537
Epoch 4, Loss: 0.002202109433710575
Epoch 5, Loss: 0.0016091568395495415
Epoch 6, Loss: 0.001220210688188672
Epoch 7, Loss: 0.0009717493085190654
Epoch 8, Loss: 0.0007853916031308472
Epoch 9, Loss: 0.0006514849374070764
Epoch 10, Loss: 0.0005507472087629139
Training complete!
